## 1. Using Langchain MCP Adapters

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")

### 1.1 Single MCP server

In [5]:
# Create server parameters for stdio connection
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent


server_params = StdioServerParameters(command="python", args=["./mcp_server_math.py"],)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)
        print(tools)

        # Create and run the agent
        agent = create_react_agent(model, tools)
        agent_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
        print(agent_response)

[StructuredTool(name='add', description='Add two numbers', args_schema={'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'addArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x10a331300>), StructuredTool(name='multiply', description='Multiply two numbers', args_schema={'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiplyArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x10a331580>)]
{'messages': [HumanMessage(content="what's (3 + 5) x 12?", additional_kwargs={}, response_metadata={}, id='cc3afdaa-1202-4acf-a4c9-d37102e5bf08'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TC3kRN2gtiSu2CF3N97XE2bk', 'functi

In [8]:
print(agent_response['messages'][-1].content)

The result of \((3 + 5) \times 12\) is 96.


### 1.2 Multiple servers at the same time

In [9]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI


config = {
        "math":{
            "command": "python",
            "args": ["./mcp_server_math.py"],
            "transport": "stdio",
            },
        "weather": {
            # make sure you start your weather server on port 8000
            "url": "http://localhost:8000/sse",
            "transport": "sse",
            }
    }

async with MultiServerMCPClient(config) as client:
    agent = create_react_agent(model, client.get_tools())
    math_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
    weather_response = await agent.ainvoke({"messages": "what is the weather in nyc?"})

print(agent_response['messages'][-1].content)
print(weather_response['messages'][-1].content)

The result of \((3 + 5) \times 12\) is 96.
The weather in NYC is always sunny!


### 1.3 Trying only with config

#### 1.3.1 single config

In [6]:
from dotenv import load_dotenv
load_dotenv()

import os
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4o-mini')


config = {
    "github": {
                "command": "npx",
                "args": [
                    "-y",
                    "@modelcontextprotocol/server-github"
                ],
                "env": {
                    "GITHUB_PERSONAL_ACCESS_TOKEN": os.getenv('GITHUB_PERSONAL_TOKEN')
                }
            }
  }


async with MultiServerMCPClient(config) as client:
    print(client.get_tools())
    agent = create_react_agent(model, client.get_tools())
    response = await agent.ainvoke({'messages':"What are the things you can do with my repository"})
    print(response['messages'][-1].content)


[StructuredTool(name='create_or_update_file', description='Create or update a single file in a GitHub repository', args_schema={'type': 'object', 'properties': {'owner': {'type': 'string', 'description': 'Repository owner (username or organization)'}, 'repo': {'type': 'string', 'description': 'Repository name'}, 'path': {'type': 'string', 'description': 'Path where to create/update the file'}, 'content': {'type': 'string', 'description': 'Content of the file'}, 'message': {'type': 'string', 'description': 'Commit message'}, 'branch': {'type': 'string', 'description': 'Branch to create/update the file in'}, 'sha': {'type': 'string', 'description': 'SHA of the file being replaced (required when updating existing files)'}}, 'required': ['owner', 'repo', 'path', 'content', 'message', 'branch'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_too

/var/folders/14/v6n289_j38zblcgwvg271f1w0000gn/T/ipykernel_57328/2129254800.py:29: RuntimeWarning: coroutine 'Pregel.ainvoke' was never awaited
  response = await agent.ainvoke({'messages':"What are the things you can do with my repository"})


In [10]:
from dotenv import load_dotenv
load_dotenv()

import os
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4o-mini')


config = {
    "playwright": {
          "command": "npx",
          "args": ["@playwright/mcp@latest"],
          "env": {
            "DISPLAY": ":1"
          }
        },
    "github": {
                "command": "npx",
                "args": [
                    "-y",
                    "@modelcontextprotocol/server-github"
                ],
                "env": {
                    "GITHUB_PERSONAL_ACCESS_TOKEN": os.getenv('GITHUB_PERSONAL_TOKEN')
                }
            }
  }


async with MultiServerMCPClient(config) as client:
    print(client.get_tools())
    agent = create_react_agent(model, client.get_tools())
    response = await agent.ainvoke({'messages':'What are the tools you have?'})
    print(response['messages'][-1].content)

[StructuredTool(name='browser_close', description='Close the page', args_schema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x11bc4c860>), StructuredTool(name='browser_wait', description='Wait for a specified time in seconds', args_schema={'type': 'object', 'properties': {'time': {'type': 'number', 'description': 'The time to wait in seconds'}}, 'required': ['time'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x11bc4c5e0>), StructuredTool(name='browser_resize', description='Resize the browser window', args_schema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type

/var/folders/14/v6n289_j38zblcgwvg271f1w0000gn/T/ipykernel_57328/2867987185.py:36: RuntimeWarning: coroutine 'Pregel.ainvoke' was never awaited
  response = await agent.ainvoke({'messages':'What are the tools you have?'})
